# Portfolio Risk & Pairs Trading Engine

## 1) Portfolio backtest demo

In [ ]:
import pandas as pd, matplotlib.pyplot as plt
from core.io import load_prices_csv
from core.backtest import run_backtest

prices = load_prices_csv("../data/prices_small.csv")
kpis = run_backtest(prices, {"AAA":0.5,"BBB":0.5}, {"AAA":0.5,"BBB":0.5}, window=2, step=1)
kpis


## 2) Pairs demo (Yahoo Finance cached)

In [ ]:
from core.data import fetch_or_cache_yf
from strategies.pairs_kalman import Pair, KalmanPairsTrader
import numpy as np
import matplotlib.pyplot as plt

yf_df = fetch_or_cache_yf(["AAPL","MSFT","SPY"], period="1y", interval="1d")
aapl = yf_df[yf_df.ticker=="AAPL"].set_index("date")["adj_close"]
msft = yf_df[yf_df.ticker=="MSFT"].set_index("date")["adj_close"]
spy  = yf_df[yf_df.ticker=="SPY" ].set_index("date")["adj_close"]

spy_ret = spy.pct_change().fillna(0.0).to_numpy()

pair = Pair("AAPL","MSFT", aapl, msft)
trader = KalmanPairsTrader(open_z=1.5, close_z=0.4)
opens, closes = trader.trade_signals(pair, spy_ret)

spread = pair.spread(spy_ret, trader.gamma)

plt.figure(figsize=(7,4))
plt.plot(spread, label="Residual spread")
for i in opens:  plt.axvline(i, linestyle="--", alpha=0.4)
for i in closes: plt.axvline(i, linestyle=":", alpha=0.4)
plt.title("Pairs residual spread with open/close markers")
plt.legend(); plt.show()
